# Refine in `load_summarize_chain`

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
from urllib.request import urlopen
import html
import gradio as gr
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
import tiktoken
from pprint import pprint

QUESTION_TEMPLATE_TXT = """Write a detailed summary (in bullet points, using "-" for bullets) of the following:

"{text}"

CONCISE SUMMARY:"""

REFINE_TEMPLATE_TXT = """Your job is to produce a final summary in bullet points (using "-" for bullets).
You are provided an existing summary here:
<existing_summary>
{existing_answer}
</existing_summary>

You are provided new text.
<new_text>
{text}
</new_text>

Given the new text, refine the original summary.
If the context isn't useful, return the original summary. Answer your summary only, not other texts.
"""

/Users/pisek/miniconda3/envs/py3p12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("sample-text.txt", "r") as f:
    transcript = f.read()

In [3]:
pprint(transcript)

("think it's possible that physics has exploits and we should be trying to "
 'find them arranging some kind of a crazy quantum mechanical system that '
 'somehow gives you buffer overflow somehow gives you a rounding error in the '
 'floating Point synthetic intelligences are kind of like the next stage of '
 "development and I don't know where it leads to like at some point I suspect "
 'the universe is some kind of a puzzle these synthetic AIS will uncover that '
 'puzzle and solve it the following is a conversation with Andre capothy '
 'previously the director of AI at Tesla and before that at open Ai and '
 'Stanford he is one of the greatest scientists engineers and Educators in the '
 'history of artificial intelligence this is the Lex Friedman podcast to '
 "support it please check out our sponsors and now dear friends here's Andre "
 'capathi what is a neural network and why does it seem to uh do such a '
 "surprisingly good job of learning what is a neural network it's a "
 

# Helper function

In [4]:
def get_text_splitter(chunk_size: int, overlap_size: int):
    return RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=overlap_size)

def convert_text_to_tokens(text, encoder="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(encoder)
    return enc.encode(text)

# Prepare docs

In [5]:
transcript_up_to = 50000

chunk_size = 1000 # this is in tokens
overlap_size = 0 # this is in tokens

### CODE

docs = [Document(
    page_content=transcript[:transcript_up_to],
    # metadata={"source": url}
)]

text_splitter = get_text_splitter(chunk_size=chunk_size, overlap_size=overlap_size)
split_docs = text_splitter.split_documents(docs)

n_docs = len(split_docs)

print("number of docs: ", n_docs)

# Count characters in each document
char_counts = [len(doc.page_content) for doc in split_docs]
print(f"Characters counts per document: {char_counts}")
print(f"Total Characters across {n_docs} documents: {sum(char_counts)}")
# split_docs
  # Using gpt-3.5-turbo tokenizer as closest approximation for llama models
token_counts = [len(convert_text_to_tokens(doc.page_content)) for doc in split_docs]
print(f"Token counts per document: {token_counts}")
print(f"Total tokens across {n_docs} documents: {sum(token_counts)}")
# Count words in each document
word_counts = [len(doc.page_content.split()) for doc in split_docs]
print(f"Word counts per document: {word_counts}")
print(f"Total words across {n_docs} documents: {sum(word_counts)}")


number of docs:  10
Characters counts per document: [5012, 5166, 4949, 5018, 4940, 5034, 4956, 5178, 5217, 4521]
Total Characters across 10 documents: 49991
Token counts per document: [988, 994, 996, 1007, 999, 1002, 990, 986, 1000, 903]
Total tokens across 10 documents: 9865
Word counts per document: [906, 943, 925, 950, 921, 942, 908, 928, 938, 867]
Total words across 10 documents: 9228


# LLM

In [6]:
model = "llama3.2"
base_url = "http://localhost:11434"
temperature = 0.5
num_ctx = 2048 # number of context length, DEFAULT = 2048
# https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html#langchain_ollama.llms.OllamaLLM.num_ctx
num_predict = 256 # number of tokens to predict, Default: 128, -1 = infinite generation, -2 = fill context
# https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html#langchain_ollama.llms.OllamaLLM.num_predict
# more info: https://python.langchain.com/api_reference/ollama/llms/langchain_ollama.llms.OllamaLLM.html#ollamallm

llm = ChatOllama(
        model=model,
        base_url=base_url,
        temperature=temperature,
        num_ctx=num_ctx,
        num_predict=num_predict,
        format='',
        verbose=True
)


# Run `load_summarize_chain` naively

In [7]:
chain_type = "refine"

refine_prompt_txt = REFINE_TEMPLATE_TXT
question_prompt_text = QUESTION_TEMPLATE_TXT

refine_prompt = PromptTemplate(
    template=refine_prompt_txt,
    input_variables=["text"]
)

question_prompt = PromptTemplate(
    template=question_prompt_text,
    input_variables=["text"]
)

chain = load_summarize_chain(
            llm, 
            chain_type=chain_type,
            question_prompt=question_prompt,
            refine_prompt=refine_prompt,
            verbose=True
)   
output = chain.invoke(split_docs[:5])



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a detailed summary (in bullet points, using "-" for bullets) of the following:

"think it's possible that physics has exploits and we should be trying to find them arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow somehow gives you a rounding error in the floating Point synthetic intelligences are kind of like the next stage of development and I don't know where it leads to like at some point I suspect the universe is some kind of a puzzle these synthetic AIS will uncover that puzzle and solve it the following is a conversation with Andre capothy previously the director of AI at Tesla and before that at open Ai and Stanford he is one of the greatest scientists engineers and Educators in the history of artificial intelligence this is the Lex Friedman podcast to support it please check out our sponsors and now dear friends here's A

In [8]:
pprint(output)

{'input_documents': [Document(metadata={}, page_content="think it's possible that physics has exploits and we should be trying to find them arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow somehow gives you a rounding error in the floating Point synthetic intelligences are kind of like the next stage of development and I don't know where it leads to like at some point I suspect the universe is some kind of a puzzle these synthetic AIS will uncover that puzzle and solve it the following is a conversation with Andre capothy previously the director of AI at Tesla and before that at open Ai and Stanford he is one of the greatest scientists engineers and Educators in the history of artificial intelligence this is the Lex Friedman podcast to support it please check out our sponsors and now dear friends here's Andre capathi what is a neural network and why does it seem to uh do such a surprisingly good job of learning what is a neural network it'

In [9]:
pprint(output["output_text"])

('- The discussion expands on the possibility of intelligent alien '
 'civilizations, with a focus on the challenges of detecting or communicating '
 'with extraterrestrial life and the potential for rapid progress events in '
 'the evolution of life and intelligence.\n'
 '\n'
 '- The author questions the difficulty of the origin of life on Earth, '
 'suggesting that it may have arisen from basic chemistry and could be '
 'considered as part of a larger scientific experiment.\n'
 '\n'
 '- The idea is proposed that humanity might be part of a simulation or code, '
 'with synthetic intelligences potentially uncovering the underlying puzzle '
 'and solving it in the future.\n'
 '\n'
 "- The author reflects on the concept of a book-like Earth, where life's "
 'story can be summarized in a few sentences or paragraphs, and considers the '
 'possible endings and terminating conditions for human civilization.\n'
 '\n'
 '- The discussion touches on the idea that humanity is an inefficient '
 'b

In [10]:
print("Token counts: {}".format(len(convert_text_to_tokens(output["output_text"]))))

Token counts: 218


# Effects of `num_predict`

In [11]:
num_ctx = 1024*3
num_predict = 1024

llm = ChatOllama(
        model=model,
        base_url=base_url,
        temperature=temperature,
        num_ctx=num_ctx,
        num_predict=num_predict,
        format='',
        verbose=True
)

chain = load_summarize_chain(
            llm, 
            chain_type=chain_type,
            question_prompt=question_prompt,
            refine_prompt=refine_prompt,
            verbose=True
)   
output = chain.invoke(split_docs)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a detailed summary (in bullet points, using "-" for bullets) of the following:

"think it's possible that physics has exploits and we should be trying to find them arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow somehow gives you a rounding error in the floating Point synthetic intelligences are kind of like the next stage of development and I don't know where it leads to like at some point I suspect the universe is some kind of a puzzle these synthetic AIS will uncover that puzzle and solve it the following is a conversation with Andre capothy previously the director of AI at Tesla and before that at open Ai and Stanford he is one of the greatest scientists engineers and Educators in the history of artificial intelligence this is the Lex Friedman podcast to support it please check out our sponsors and now dear friends here's A

In [12]:
pprint(output['output_text'])

('- The discussion continues to explore the idea of simulating life in '
 'computer programs, with the speaker suggesting that running a simulation '
 'could provide insights into the emergence and evolution of life.\n'
 '- They propose that synthetic intelligences may be the next stage of '
 'development, potentially uncovering secrets about the universe and solving '
 'complex problems.\n'
 '- The speaker questions whether humanity is part of a "science experiment" '
 'or if we are just NPCs (non-player characters) within a simulated reality.\n'
 '- They mention the idea of exploiting the system to break free from it, with '
 'the speaker suggesting that finding a way to do so could be the ultimate '
 'goal.\n'
 '- The conversation touches on the concept of physics being a kind of '
 'computation with potential bugs and exploits, and the possibility of '
 'discovering shortcuts or "weirdness" in the physical world.\n'
 '- The speaker suggests that a third-generation AGI (artificial g